In [ ]:
# mount drive into colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# change directory
%cd '/content/drive/MyDrive/Rahnema/Rahnema-College-Team'

/content/drive/.shortcut-targets-by-id/1mN07jhPqKcfkME6GxvaJOdBZWSwOA7AN/Rahnema-College-Team


***
# Imports

In [ ]:
%%capture
!pip install pyod
!pip install alibi-detect

In [ ]:
from alibi_detect.od import IForest

import warnings
warnings.filterwarnings('ignore')

import datetime as dt

from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator

from gensim.parsing.preprocessing import preprocess_string
from sklearn import utils
from tqdm import tqdm

import pandas as pd
import numpy as np
import re

from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
from gensim.parsing.preprocessing import preprocess_string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import utils

from matplotlib import pyplot as plt

from pyod.models.auto_encoder import AutoEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

***
# Load File

In [ ]:
df_logs = pd.read_csv('logs_cleaned.csv')
df_logs.head()

,Unnamed: 0,ip,datetime,method,request,status,size,user_agent,response_time,date,year,quarter,month,day,time,hour,minute,second,subnet,first_part_request,last_part_request,depth_request,status_group,country,browser_family,browser_version,os_family,os_version,device_family,device_brand,is_bot,id,robots,time1,id1,id2
0,0,207.213.193.143,2021-05-12 05:06:00+04:30,Get,/cdn/profiles/1026106239,304,0,Googlebot-Image/1.0,32,2021-05-12,2021,2,5,12,05:06:00,5,6,0,207.213.193,cdn,1026106239,3.0,3xx,United States,Googlebot-Image,1.0,Other,NaN,Spider,Spider,1,207.213.193.143 Googlebot-Image/1.0,0.0,05:0,207.213.193.143 Googlebot-Image/1.0-05,207.213.193.143 Googlebot-Image/1.0-05:0
1,1,207.213.193.143,2021-05-12 05:06:00+04:30,Get,images/badge.png,304,0,Googlebot-Image/1.0,4,2021-05-12,2021,2,5,12,05:06:00,5,6,0,207.213.193,images,badge.png,2.0,3xx,United States,Googlebot-Image,1.0,Other,NaN,Spider,Spider,1,207.213.193.143 Googlebot-Image/1.0,0.0,05:0,207.213.193.143 Googlebot-Image/1.0-05,207.213.193.143 Googlebot-Image/1.0-05:0
2,2,35.110.222.153,2021-05-12 05:06:00+04:30,Get,/pages/630180847,200,52567,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,32,2021-05-12,2021,2,5,12,05:06:00,5,6,0,35.110.222,pages,630180847,2.0,2xx,United States,Samsung Internet,4.0,Android,6.0.1,Samsung SM-J710GN,Samsung,0,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...,0.0,05:0,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...
3,3,35.108.208.99,2021-05-12 05:06:00+04:30,Get,images/fav_icon2.ico,200,23531,Mozilla/5.0 (Linux; Android 6.0; CAM-L21) Appl...,20,2021-05-12,2021,2,5,12,05:06:00,5,6,0,35.108.208,images,fav_icon2.ico,2.0,2xx,United States,Chrome Mobile,91.0.4472,Android,6.0,Huawei CAM-L21,Huawei,0,35.108.208.99 Mozilla/5.0 (Linux; Android 6.0;...,0.0,05:0,35.108.208.99 Mozilla/5.0 (Linux; Android 6.0;...,35.108.208.99 Mozilla/5.0 (Linux; Android 6.0;...
4,4,35.110.222.153,2021-05-12 05:06:00+04:30,Get,images/sanjagh_logo_purpule5.png,200,4680,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,8,2021-05-12,2021,2,5,12,05:06:00,5,6,0,35.110.222,images,sanjagh_logo_purpule5.png,2.0,2xx,United States,Samsung Internet,4.0,Android,6.0.1,Samsung SM-J710GN,Samsung,0,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...,0.0,05:0,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...


In [ ]:
# seperate train and test data based on time
X_train, X_test, _, _ = train_test_split(df_logs, df_logs['is_bot'], test_size=0.2, shuffle=False)

***
# Extract Features


In [ ]:
def extract_features(df_logs):

  print('Shape of original df:', df_logs.shape)

  # drop rows which are bots
  #df_logs = df_logs[df_logs['is_bot'] == 0]
  #print('Shape of df after removing bots:', df_logs.shape)

  # click rate
  df_logs['num_req'] = 1
  df_clickrate = df_logs[['id', 'num_req']].groupby(['id']).sum()

  # image
  df_logs['is_image'] = df_logs['last_part_request'].str.match('.*.(jpeg|jpg|png)', case=False).astype(int)
  df_logs['img2req'] = df_logs['is_image'] / df_logs['num_req']
  df_img2req = df_logs[['id', 'img2req']].groupby(['id']).sum()
  # font
  df_logs['is_font'] = df_logs['last_part_request'].str.match('.*.(woff)', case=False).astype(int)
  df_logs['font2req'] = df_logs['is_font'] / df_logs['num_req']
  df_font2req = df_logs[['id', 'font2req']].groupby(['id']).sum()
  # js
  df_logs['is_js'] = df_logs['last_part_request'].str.match('.*.(js)', case=False).astype(int)
  df_logs['js2req'] = df_logs['is_js'] / df_logs['num_req']
  df_js2req = df_logs[['id', 'js2req']].groupby(['id']).sum()
  # css
  df_logs['is_css'] = df_logs['last_part_request'].str.match('.*.(css)', case=False).astype(int)
  df_logs['css2req'] = df_logs['is_css'] / df_logs['num_req']
  df_css2req = df_logs[['id', 'css2req']].groupby(['id']).sum()
  # icon
  df_logs['is_ico'] = df_logs['last_part_request'].str.match('.*.(ico)', case=False).astype(int)
  df_logs['ico2req'] = df_logs['is_ico'] / df_logs['num_req']
  df_ico2req = df_logs[['id', 'ico2req']].groupby(['id']).sum()
  # text
  df_logs['is_text'] = df_logs['last_part_request'].str.match('.*.(txt)', case=False).astype(int)
  df_logs['txt2req'] = df_logs['is_text'] / df_logs['num_req']
  df_txt2req = df_logs[['id', 'txt2req']].groupby(['id']).sum()
  # file2request
  dfs = [df_img2req, df_font2req, df_js2req, df_css2req, df_ico2req, df_txt2req]
  df_file2req = pd.concat(dfs, axis=1, sort=False)

  # status
  df_logs['is_308'] = (df_logs['status'] == 308).astype(int)
  df_logs['is_408'] = (df_logs['status'] == 408).astype(int)
  df_logs['is_413'] = (df_logs['status'] == 413).astype(int)
  df_logs['is_206'] = (df_logs['status'] == 206).astype(int)
  df_logs['is_405'] = (df_logs['status'] == 405).astype(int)
  df_logs['is_499'] = (df_logs['status'] == 499).astype(int)
  df_logs['is_301'] = (df_logs['status'] == 301).astype(int)
  df_logs['is_307'] = (df_logs['status'] == 307).astype(int)
  df_logs['is_101'] = (df_logs['status'] == 101).astype(int)

  cols = ['id', 'is_308', 'is_408', 'is_413', 'is_206', 'is_405', 'is_499', 'is_301', 'is_307', 'is_101']
  df_status = df_logs[cols].groupby(['id']).sum()

  # methods
  df_logs['is_head'] = (df_logs['method'] == 'Head').astype(int)
  df_logs['is_post'] = (df_logs['method'] == 'Post').astype(int)
  df_logs['is_put'] = (df_logs['method'] == 'Put').astype(int)
  df_logs['is_get'] = (df_logs['method'] == 'Get').astype(int)

  cols = ['id', 'is_head', 'is_post', 'is_put', 'is_get']
  df_method = df_logs[cols].groupby(['id']).sum()

  # robot.txt
  df_robots = df_logs[['id', 'robots']].groupby(['id']).sum()

  # depth request
  df_depth = df_logs[['id', 'depth_request']].groupby(['id']).std()

  # hours
  df_logs['is_5'] = (df_logs['hour'] == 5).astype(int)
  df_logs['is_6'] = (df_logs['hour'] == 6).astype(int)
  df_logs['is_7'] = (df_logs['hour'] == 7).astype(int)
  df_logs['is_8'] = (df_logs['hour'] == 8).astype(int)
  df_logs['is_9'] = (df_logs['hour'] == 9).astype(int)
  df_logs['is_10'] = (df_logs['hour'] == 10).astype(int)
  df_logs['is_11'] = (df_logs['hour'] == 11).astype(int)
  df_logs['is_12'] = (df_logs['hour'] == 12).astype(int)
  df_logs['is_13'] = (df_logs['hour'] == 13).astype(int)
  df_logs['is_14'] = (df_logs['hour'] == 14).astype(int)
  df_logs['is_15'] = (df_logs['hour'] == 15).astype(int)

  cols = ['id', 'is_5', 'is_6', 'is_7', 'is_8', 'is_9', 'is_10', 'is_11', 'is_12', 'is_13', 'is_14', 'is_15']
  df_hour = df_logs[cols].groupby(['id']).sum()

  # time differentiate
  df_logs['time'] = pd.to_timedelta(df_logs['time'])
  df_logs['gap'] = (df_logs
                    .groupby('id')['time']
                    .diff()
                    .dt.floor('T')
                    .dt.total_seconds()
                    .div(60)
                    .fillna(0, downcast='infer'))
  
  df_mean_gap = df_logs[['id', 'gap']].groupby(['id']).mean()
  df_std_gap = df_logs[['id', 'gap']].groupby(['id']).std()
  df_gap = pd.concat([df_mean_gap, df_std_gap], axis=1, sort=False)
  df_gap.columns = ['mean_gap', 'std_gap']

  # response time
  df_response = df_logs[['id', 'response_time']].groupby(['id']).mean()
  
  # size
  df_size = df_logs[['id', 'size']].groupby(['id']).mean()

  # truncate request to its first 3 path
  n = 4
  df_logs['trunc_req'] = df_logs['request'].apply(lambda x: '/'.join(x.split('/', n)[:n]))
  print('Number of unique requests:', df_logs['request'].nunique())
  print('Number of unique truncated requests:', df_logs['trunc_req'].nunique())

  # Requests Embedding
  df_requests = (df_logs.groupby(['id'])['trunc_req'].apply(lambda x: ' '.join(set(x.dropna())))).to_frame()

  # merge dataframes
  dfs = [df_clickrate, df_file2req, df_status, df_method, df_robots, 
         df_depth, df_hour, df_gap, df_response, df_size, df_requests]
         
  df = pd.concat(dfs, axis=1, sort=False).reset_index()
  print('Shape of final df:', df.shape)

  # drop rows with number of requests less than 5
  num_reqs = 5
  df = df[df['num_req'] > num_reqs].reset_index(drop=True)
  print('Shape of final df with more than 5 reqs:', df.shape)

  # calculate bigram of requests
  trunc_reqs = df['trunc_req'].to_list()
  bigram_reqs = []
  
  for trunc_req in trunc_reqs:
    reqs = trunc_req.split(' ')
    len_reqs = len(reqs)
    bigram = []
    for i in range(len_reqs):
      if i == len_reqs-1:
        break
      bigram.append(reqs[i] + '#' + reqs[i+1])
    bigram_reqs.append(' '.join(bigram))

  df['bigram_req'] = bigram_reqs
  
  return df

In [ ]:
df_train = extract_features(X_train)

Shape of original df: (993556, 36)
Number of unique requests: 111493
Number of unique truncated requests: 62296
Shape of final df: (42117, 39)
Shape of final df with more than 5 reqs: (25565, 39)


In [ ]:
df_test = extract_features(X_test)

Shape of original df: (248389, 36)
Number of unique requests: 45384
Number of unique truncated requests: 27098
Shape of final df: (12278, 39)
Shape of final df with more than 5 reqs: (5104, 39)


In [ ]:
df_test.head()

,id,num_req,img2req,font2req,js2req,css2req,ico2req,txt2req,is_308,is_408,is_413,is_206,is_405,is_499,is_301,is_307,is_101,is_head,is_post,is_put,is_get,robots,depth_request,is_5,is_6,is_7,is_8,is_9,is_10,is_11,is_12,is_13,is_14,is_15,mean_gap,std_gap,response_time,size,trunc_req,bigram_req
0,102.105.101.172 Mozilla/5.0 (Linux; Android 8....,38,15.0,3.0,4.0,1.0,1.0,0.0,0,0,0,0,0,0,0,1,1,0,0,1,37,0.0,0.603451,0,0,0,0,0,0,0,0,0,38,0,0.157895,0.973329,10286.447368,50558.868421,/cdn/profiles/1781232532 images/sanjagh_logo_p...,/cdn/profiles/1781232532#images/sanjagh_logo_p...
1,102.105.139.132 Mozilla/5.0 (Windows NT 10.0; ...,800,7.0,3.0,3.0,2.0,1.0,0.0,0,0,0,0,0,0,1,0,388,0,1,389,410,0.0,0.539954,0,0,0,0,0,0,0,0,240,488,72,0.022500,0.148396,81045.293750,3052.973750,images/sanjagh_logo_purpule5.png images/badge....,images/sanjagh_logo_purpule5.png#images/badge....
2,102.105.179.61 Mozilla/5.0 (Windows NT 10.0; W...,42,14.0,3.0,4.0,1.0,2.0,0.0,0,0,0,0,0,0,0,1,1,0,0,1,41,0.0,0.739228,0,0,0,0,0,0,0,0,0,42,0,0.023810,0.154303,2210.904762,46781.357143,/cdn/profiles/1781232532 images/sanjagh_logo_p...,/cdn/profiles/1781232532#images/sanjagh_logo_p...
3,102.105.199.201 Mozilla/5.0 (Linux; Android 6....,22,6.0,3.0,3.0,2.0,2.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,22,0.0,0.476731,0,0,0,0,0,0,0,0,22,0,0,0.000000,0.000000,13.636364,42125.409091,images/sanjagh_logo_purpule5.png images/aparat...,images/sanjagh_logo_purpule5.png#images/aparat...
4,102.105.22.17 Mozilla/5.0 (Linux; Android 9; R...,14,6.0,3.0,3.0,2.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,14,0.0,0.267261,0,0,0,0,0,0,0,0,0,14,0,0.000000,0.000000,4.000000,0.000000,css/page.2f0fc69390da8cdff683.css fonts/IRANSa...,css/page.2f0fc69390da8cdff683.css#fonts/IRANSa...


***
# Doc2Vec

In [ ]:
def doc2vec(df):
  # tokenize and tag the card text
  requests_docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df['bigram_req'])]
  # instantiate model
  model = Doc2Vec(vector_size=100, min_count=2, workers=8, epochs = 20)
  # build vocab
  model.build_vocab(requests_docs)
  # train model
  model.train(requests_docs, total_examples=model.corpus_count, epochs=model.epochs)

  # generate vectors
  request2vec = [model.infer_vector((df['bigram_req'][i].split(' '))) for i in range(0,len(df['bigram_req']))]

  # create a list of lists
  dtv = np.array(request2vec).tolist()
  # set list to dataframe column
  df['request2vec'] = dtv

  # separate each list item into a new column
  df_req2vecs = pd.DataFrame(df['request2vec'].tolist(), index= df.index)

  return df_req2vecs

In [ ]:
# embdedding features
df_train_embed = doc2vec(df_train)
df_test_embed = doc2vec(df_test)

In [ ]:
# statistical features
df_train_stat = df_train.drop(['id', 'trunc_req', 'bigram_req', 'request2vec'], axis=1)
cols_train = df_train_stat.columns
df_train_stat = StandardScaler().fit_transform(df_train_stat)
df_train_stat = pd.DataFrame(df_train_stat, columns=cols_train)

df_test_stat = df_test.drop(['id', 'trunc_req', 'bigram_req', 'request2vec'], axis=1)
cols_test = df_test_stat.columns
df_test_stat = StandardScaler().fit_transform(df_test_stat)
df_test_stat = pd.DataFrame(df_test_stat, columns=cols_test)

In [ ]:
# all features
df_train_all = df_train_embed.join(df_train_stat)
df_test_all = df_test_embed.join(df_test_stat)

***
# Isolation Forest - using ALIBI Package

In [ ]:
def iforest(X_train, X_test):
  od = IForest(threshold=0., n_estimators=100)
  od.fit(X_train)
  od.infer_threshold(X_test, threshold_perc=95)
  preds = od.predict(X_test, return_instance_score=True)
  scores = preds['data']['instance_score']
  least_scores = np.argsort(scores)[-20:]
  return least_scores

In [ ]:
# just embedding
least_scores_embed = iforest(df_train_embed, df_test_embed)
df_test.iloc[least_scores_embed][['id']]

,id
2359,35.136.16.103 Mozilla/5.0 (Linux; Android 8.1....
1726,244.251.234.120 Mozilla/5.0 (Windows NT 6.3; W...
2804,35.216.8.86 Mozilla/5.0 (Android 9; Mobile; rv...
3653,35.82.215.161 Mozilla/5.0 (Linux; Android 10; ...
3542,35.68.3.202 Mozilla/5.0 (Android 8.1.0; Mobile...
4954,92.130.216.88 Mozilla/5.0 (Linux; Android 4.4....
3210,35.33.20.165 Mozilla/5.0 (Linux; Android 10; S...
2111,35.116.130.215 Mozilla/5.0 (Windows NT 6.1) Ap...
1421,207.213.207.224 Googlebot-Image/1.0
4407,64.250.202.229 Opera/9.80 (J2ME/MIDP; Opera Mi...


In [ ]:
# just statistical
least_scores_stat = iforest(df_train_stat, df_test_stat)
df_test.iloc[least_scores_stat][['id']]

,id
1522,217.14.27.175 Mozilla/5.0 (Linux; Android 9; S...
516,14.212.130.18 Mozilla/5.0 (Windows NT 10.0; Wi...
3210,35.33.20.165 Mozilla/5.0 (Linux; Android 10; S...
3853,37.143.185.123 Mozilla/5.0 (Linux; Android 10;...
2318,35.125.246.182 Mozilla/5.0 (Linux; Android 9; ...
1341,200.88.14.217 Mozilla/5.0 (Linux; Android 9; J...
1,102.105.139.132 Mozilla/5.0 (Windows NT 10.0; ...
3326,35.40.33.82 Mozilla/5.0 (Linux; Android 8.0.0;...
2100,35.111.55.89 Mozilla/5.0 (Windows NT 10.0; Win...
986,148.208.107.7 Mozilla/5.0 (Windows NT 10.0; Wi...


In [ ]:
# both statistics and embedding
least_scores_all = iforest(df_train_all, df_test_all)
df_test.iloc[least_scores_all][['id']]

,id
4407,64.250.202.229 Opera/9.80 (J2ME/MIDP; Opera Mi...
1412,207.213.207.158 Mozilla/5.0 (Linux; Android 6....
1260,20.173.183.139 Mozilla/5.0 (Windows NT 10.0; W...
3898,37.171.180.102 Mozilla/5.0 (iPhone; CPU iPhone...
664,14.6.56.52 Mozilla/5.0 (Windows NT 6.1) AppleW...
1666,228.83.29.144 Mozilla/5.0 (Linux; Android 5.1....
530,14.212.55.74 Mozilla/5.0 (Linux; Android 11; S...
1121,155.11.137.225 Mozilla/5.0 (Windows NT 6.1) Ap...
2100,35.111.55.89 Mozilla/5.0 (Windows NT 10.0; Win...
2318,35.125.246.182 Mozilla/5.0 (Linux; Android 9; ...


In [ ]:
# save results to csv
df_test.iloc[least_scores_embed][['id']].to_csv('iforest_anomalies_embed_old.csv')
df_test.iloc[least_scores_stat][['id']].to_csv('iforest_anomalies_stat_old.csv')
df_test.iloc[least_scores_all][['id']].to_csv('iforest_anomalies_all_old.csv')

***
# Auto-encoder using PyOD

In [ ]:
def auto_encoder(X_train, X_test, hd):
  # define model
  clf = AutoEncoder(batch_size = 64, 
                    contamination = 0.1, 
                    dropout_rate = 0.2, 
                    epochs = 50,
                    hidden_activation = 'relu', 
                    hidden_neurons = hd,
                    l2_regularizer = 0.1,
                    loss = 'mean_squared_error',
                    optimizer = 'adam', 
                    output_activation = 'sigmoid', 
                    preprocessing = True,
                    random_state = None, 
                    validation_size = 0.1, 
                    verbose = 1)
  
  # shuffle data
  X_train = X_train.sample(frac = 1)

  # train the model
  clf.fit(X_train)

  # Predict the anomaly scores
  y_test_scores = clf.decision_function(X_test)  # outlier scores
  y_test_scores = pd.Series(y_test_scores)

  df = X_test.copy()
  df['score'] = y_test_scores
  
  return df

In [ ]:
hd = [100, 50, 12, 50, 100]
th = 37
df_embed = auto_encoder(df_train_embed, df_test_embed, hd)
df_embed['cluster'] = np.where(df_embed['score'] < th, 0, 1)
df_embed['cluster'].value_counts()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_9 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)               

0    5085
1      19
Name: cluster, dtype: int64

In [ ]:
pd.DataFrame(df_test[df_embed['cluster'] == 1]['id'])

,id
530,14.212.55.74 Mozilla/5.0 (Linux; Android 11; S...
664,14.6.56.52 Mozilla/5.0 (Windows NT 6.1) AppleW...
1176,155.193.28.169 Mozilla/5.0 (Linux; Android 7.1...
1260,20.173.183.139 Mozilla/5.0 (Windows NT 10.0; W...
1390,207.213.207.102 Googlebot-Image/1.0
1396,207.213.207.116 Googlebot-Image/1.0
1401,207.213.207.130 Googlebot-Image/1.0
1406,207.213.207.144 Googlebot-Image/1.0
1410,207.213.207.158 Googlebot-Image/1.0
1415,207.213.207.172 Googlebot-Image/1.0


In [ ]:
hd = [39, 25, 12, 25, 39]
#hd = [59, 40, 12, 40, 59]
th = 45
df_stat = auto_encoder(df_train_stat, df_test_stat, hd)
df_stat['cluster'] = np.where(df_stat['score'] < th, 0, 1)
df_stat['cluster'].value_counts()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 37)                1406      
_________________________________________________________________
dropout_28 (Dropout)         (None, 37)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 37)                1406      
_________________________________________________________________
dropout_29 (Dropout)         (None, 37)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 39)                1482      
_________________________________________________________________
dropout_30 (Dropout)         (None, 39)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 25)               

0    5090
1      14
Name: cluster, dtype: int64

In [ ]:
pd.DataFrame(df_test[df_stat['cluster'] == 1]['id'])

,id
986,148.208.107.7 Mozilla/5.0 (Windows NT 10.0; Wi...
1140,155.114.143.38 MoziIIa/5.0 (X11; Linux x86_64)...
1332,20.92.247.146 sentry/21.4.1 (https://sentry.io)
1390,207.213.207.102 Googlebot-Image/1.0
1396,207.213.207.116 Googlebot-Image/1.0
1401,207.213.207.130 Googlebot-Image/1.0
1406,207.213.207.144 Googlebot-Image/1.0
3116,35.26.221.84 Mozilla/5.0 (Windows NT 6.1; Win6...
3823,36.67.23.210 Go-http-client/2.0
4388,60.148.0.167 Go-http-client/2.0


In [ ]:
hd = [139, 80, 30, 12, 30, 80, 139]
#hd = [159, 80, 30, 12, 30, 80, 159]
th = 50
df_all = auto_encoder(df_train_all, df_test_all, hd)
df_all['cluster'] = np.where(df_all['score'] < th, 0, 1)
df_all['cluster'].value_counts()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             (None, 137)               18906     
_________________________________________________________________
dropout_49 (Dropout)         (None, 137)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 137)               18906     
_________________________________________________________________
dropout_50 (Dropout)         (None, 137)               0         
_________________________________________________________________
dense_58 (Dense)             (None, 139)               19182     
_________________________________________________________________
dropout_51 (Dropout)         (None, 139)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 80)               

0    5087
1      17
Name: cluster, dtype: int64

In [ ]:
pd.DataFrame(df_test[df_all['cluster'] == 1]['id'])

,id
986,148.208.107.7 Mozilla/5.0 (Windows NT 10.0; Wi...
1140,155.114.143.38 MoziIIa/5.0 (X11; Linux x86_64)...
1332,20.92.247.146 sentry/21.4.1 (https://sentry.io)
1390,207.213.207.102 Googlebot-Image/1.0
1396,207.213.207.116 Googlebot-Image/1.0
1401,207.213.207.130 Googlebot-Image/1.0
1406,207.213.207.144 Googlebot-Image/1.0
1410,207.213.207.158 Googlebot-Image/1.0
1415,207.213.207.172 Googlebot-Image/1.0
1418,207.213.207.186 Googlebot-Image/1.0


In [ ]:
# save results to csv
pd.DataFrame(df_test[df_embed['cluster'] == 1]['id']).to_csv('autoencoder_anomalies_embed_old.csv')
pd.DataFrame(df_test[df_stat['cluster'] == 1]['id']).to_csv('autoencoder_anomalies_stat_old.csv')
pd.DataFrame(df_test[df_all['cluster'] == 1]['id']).to_csv('autoencoder_anomalies_all_old.csv')